<a href="https://colab.research.google.com/github/SteBaum/Nigeria-Bank/blob/main/Yolov5_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Connect to Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Convert all images into png format
import glob
for item in glob.glob("*"):
 Path(str(item)).rename(Path(str(item)).with_suffix('.png'))


In [ ]:
# Check the number of files in folder
len(glob.glob("*"))

1086

In [ ]:
# Import modules
import glob
from pathlib import Path
import shutil
from tqdm import tqdm
import numpy as np
import json
import cv2
from IPython.display import display
from sklearn.model_selection import train_test_split
import seaborn as sns
import PIL.Image as Image
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
import ast
import os
import re


In [ ]:
# Load csv dataframe
bankreports = pd.read_csv("/content/gdrive/MyDrive/Liberia_bank/via_project_15Feb2022_6h10m_csv.csv")
bankreports.shape

(146, 7)

In [ ]:
type(bankreports_new_images.filename[1])

str

In [ ]:
# Convert all .jpg names into .png
bankreports.filename = [item.replace(".jpg", ".png") for item in bankreports.filename]

In [ ]:
bankreports.head()

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,1-2.png,144789,{},2,0,"{""name"":""rect"",""x"":20,""y"":136,""width"":200,""hei...",{}
1,1-2.png,144789,{},2,1,"{""name"":""rect"",""x"":276,""y"":39,""width"":201,""hei...",{}
2,1-2-pages.png,120719,{},2,0,"{""name"":""rect"",""x"":14,""y"":200,""width"":220,""hei...",{}
3,1-2-pages.png,120719,{},2,1,"{""name"":""rect"",""x"":15,""y"":107,""width"":219,""hei...",{}
4,Addiko-bank-st.png,187754,{},2,0,"{""name"":""rect"",""x"":36,""y"":377,""width"":393,""hei...",{}


In [ ]:
# Check the region_shape_attributes column
bankreports.filename[97]

'Azerbaijan-Access-Bank-English-language-statement-template-in-Excel-and-PDF-format.jpg'

In [ ]:
# Seperate the region_shape_attributes into different columns
df = pd.json_normalize(bankreports.region_shape_attributes.apply(ast.literal_eval))

In [ ]:
# Join the newly created columns to the dataframe
bankreports = bankreports.join(df)

In [ ]:
# Split the images (and not annotations) into train and validation samples, training sample being 90%
train_940, val_940 = train_test_split(list(bankreports['filename'].drop_duplicates()), test_size=0.1)
len(train_940), len(val_940)

(131, 15)

In [ ]:
cd /content/gdrive/MyDrive/Liberia_bank

/content/gdrive/MyDrive/Liberia_bank


In [ ]:
# Define a function that creaets the dataset with
# the following images and text annotation files.

def create_dataset(reports, dataset_type, source_path, destination_folder):
  images_path = Path(f"{destination_folder}/images/{dataset_type}")
  images_path.mkdir(parents=True, exist_ok=True) # path and folder for image file

  labels_path = Path(f"{destination_folder}/labels/{dataset_type}")
  labels_path.mkdir(parents=True, exist_ok=True) # path and folder for annoation files

  for item in reports:
    # iterate over rows in the dataframe
    try:    
      abs_path = os.path.join(source_path, item) #create path
            
  # Open the image files and save them in the corresponding folders
      img = Image.open(abs_path)
      #img = img.convert("RGB")
      img.save(str(images_path / item), "png")
      im = cv2.imread(abs_path) # We need this module in order to get the image shape.
      h, w, _ = im.shape # Get the image height and width for the normalization.
      label_name = item.replace(".png", ".txt")
      # Open a text file with the annotation coordinates
      with (labels_path / label_name).open("w") as label_file:
        for idx, col in bankreports.iterrows():
          if item == col[0]:
            x = bankreports.iloc[idx]["x"]
            y = bankreports.iloc[idx]["y"]
            width = bankreports.iloc[idx]["width"]
            height = bankreports.iloc[idx]["height"]

          # Write the annotation coordinates into a text file
            label_file.write(f"0 {(x + width / 2)/w} {(y + height / 2)/h} {width/w} {height/h}\n")
            
    except: # accept the error and continue
      print('error', item)
      continue


In [ ]:
# Create two datasets
create_dataset(train_940, 'train','/content/gdrive/MyDrive/Liberia_bank/dataset_feb', 'png_reports')
create_dataset(val_940, 'val', '/content/gdrive/MyDrive/Liberia_bank/dataset_feb', 'png_reports')

In [ ]:
cd /content/gdrive/MyDrive/Liberia_bank/png_reports/images/val


/content/gdrive/MyDrive/Liberia_bank/png_reports/images/val


In [ ]:
# Check the number of images or annotation files in each folder
len(glob.glob("*"))

109

In [ ]:
# function that injects the new images and lables in an already existing dataset

def inject_dataset(reports, dataset_type, source_path, destination_folder):
  images_path = Path(f"{destination_folder}/images/{dataset_type}")

  labels_path = Path(f"{destination_folder}/labels/{dataset_type}")

  for item in reports:
    # iterate over rows in the dataframe
    try:    
      abs_path = os.path.join(source_path, item) #create path
            
  # Open the image files and save them in the corresponding folders
      img = Image.open(abs_path)
      #img = img.convert("RGB")
      img.save(str(images_path / item), "png")
      im = cv2.imread(abs_path) # We need this module in order to get the image shape.
      h, w, _ = im.shape # Get the image height and width for the normalization.
      label_name = item.replace(".png", ".txt")
      # Open a text file with the annotation coordinates
      with (labels_path / label_name).open("w") as label_file:
        for idx, col in bankreports.iterrows():
          if item == col[0]:
            x = bankreports.iloc[idx]["x"]
            y = bankreports.iloc[idx]["y"]
            width = bankreports.iloc[idx]["width"]
            height = bankreports.iloc[idx]["height"]

          # Write the annotation coordinates into a text file
            label_file.write(f"0 {(x + width / 2)/w} {(y + height / 2)/h} {width/w} {height/h}\n")
            
    except: # accept the error and continue
      print('error', item)
      continue

In [ ]:
inject_dataset(train_940, 'train','/content/gdrive/MyDrive/Liberia_bank/dataset_feb', 'png_reports')
inject_dataset(val_940, 'val', '/content/gdrive/MyDrive/Liberia_bank/dataset_feb', 'png_reports')

In [ ]:
cd /content/gdrive/MyDrive/Liberia_bank/png_reports/images/val

In [ ]:
# Check the number of images or annotation files in each folder
len(glob.glob("*"))